In [1]:
!pip install fairseq2
!pip install pydub sentencepiece 
!pip install git+https://github.com/facebookresearch/seamless_communication.git

  Cloning https://github.com/facebookresearch/seamless_communication.git to /tmp/pip-req-build-lejadjp7
  Running command git clone --filter=blob:none --quiet https://github.com/facebookresearch/seamless_communication.git /tmp/pip-req-build-lejadjp7
  Resolved https://github.com/facebookresearch/seamless_communication.git to commit 90e2b57ac4d82fa2bfaa25caeffe39ceb8b2ebec
  Running command git submodule update --init --recursive -q
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [2]:
!pip install demucs

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 16.1 MB/s eta 0:00:0000:0100:01
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 87.1/87.1 kB 2.5 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.6/59.6 kB 3.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 249.7/249.7 kB 10.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.8 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 96.8 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 77.4 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 39.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.6 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━

In [2]:
import io
import json
import matplotlib as mpl
import matplotlib.pyplot as plt
import mmap
import numpy
import soundfile
import torchaudio
import torch

from collections import defaultdict
from IPython.display import Audio, display
from pathlib import Path
from pydub import AudioSegment

from seamless_communication.inference import Translator
from seamless_communication.streaming.dataloaders.s2tt import SileroVADSilenceRemover

In [3]:
!wget "https://d11ywzt2xtszji.cloudfront.net/SeamlessExpressive.tar.gz?Policy=eyJTdGF0ZW1lbnQiOlt7InVuaXF1ZV9oYXNoIjoiNXdrajFuZ21maHppMTZja2Zlemdwb25mIiwiUmVzb3VyY2UiOiJodHRwczpcL1wvZDExeXd6dDJ4dHN6amkuY2xvdWRmcm9udC5uZXRcLyoiLCJDb25kaXRpb24iOnsiRGF0ZUxlc3NUaGFuIjp7IkFXUzpFcG9jaFRpbWUiOjE3NjA1Mjc0ODh9fX1dfQ__&Signature=QUwbSP8hwjLMNSyKl6O6Nil3U8PdLGsPzxdgbMUR9F2qN-K6YapCmg2DOkpBoU7TxEAO9LIDPX2MGC2dyd2hhgAkhBZSFC-aDpuZ%7E19A2QgkFTxZBddeNdQ14PbF%7E3RSNCDBzB8I-FiCr3NQZbYhGO9zkUptUbLWyfKtFQy1co8liJ6qGDpnwZ%7Ex03B0Nz9xzla3vrYWUN1qRKEAG6GqypBxNHnfmYpnwsim2ss%7EJhw-fr5uQaWakV8BltGmy0z%7EsOeuoYrHAbXW0Daz50SlzZo1uYJa7iHgcfMY6qiTIlpmntzfieRTR0v%7EvdoMpqyUl2D2SkgCz4K38JergXyUBA__&Key-Pair-Id=K15QRJLYKIFSLZ&Download-Request-ID=4264330077158465" -O /content/SeamlessExpressive.tar.gz

!tar -xzvf /content/SeamlessExpressive.tar.gz

--2025-10-13 11:25:31--  https://d11ywzt2xtszji.cloudfront.net/SeamlessExpressive.tar.gz?Policy=eyJTdGF0ZW1lbnQiOlt7InVuaXF1ZV9oYXNoIjoiNXdrajFuZ21maHppMTZja2Zlemdwb25mIiwiUmVzb3VyY2UiOiJodHRwczpcL1wvZDExeXd6dDJ4dHN6amkuY2xvdWRmcm9udC5uZXRcLyoiLCJDb25kaXRpb24iOnsiRGF0ZUxlc3NUaGFuIjp7IkFXUzpFcG9jaFRpbWUiOjE3NjA1Mjc0ODh9fX1dfQ__&Signature=QUwbSP8hwjLMNSyKl6O6Nil3U8PdLGsPzxdgbMUR9F2qN-K6YapCmg2DOkpBoU7TxEAO9LIDPX2MGC2dyd2hhgAkhBZSFC-aDpuZ%7E19A2QgkFTxZBddeNdQ14PbF%7E3RSNCDBzB8I-FiCr3NQZbYhGO9zkUptUbLWyfKtFQy1co8liJ6qGDpnwZ%7Ex03B0Nz9xzla3vrYWUN1qRKEAG6GqypBxNHnfmYpnwsim2ss%7EJhw-fr5uQaWakV8BltGmy0z%7EsOeuoYrHAbXW0Daz50SlzZo1uYJa7iHgcfMY6qiTIlpmntzfieRTR0v%7EvdoMpqyUl2D2SkgCz4K38JergXyUBA__&Key-Pair-Id=K15QRJLYKIFSLZ&Download-Request-ID=4264330077158465
Resolving d11ywzt2xtszji.cloudfront.net (d11ywzt2xtszji.cloudfront.net)... 99.84.118.34, 99.84.118.42, 99.84.118.15, ...
Connecting to d11ywzt2xtszji.cloudfront.net (d11ywzt2xtszji.cloudfront.net)|99.84.118.34|:443... connected.
HTTP reques

In [1]:
import os

# Paths
input_video = "/kaggle/input/sample-video/INPUT VEDIO 2.mp4"  # Change to your actual video file path
output_dir = "/kaggle/working/OUTPUT"
os.makedirs(output_dir, exist_ok=True)

output_audio = os.path.join(output_dir, "output_audio.wav")
output_video = os.path.join(output_dir, "output_video.mp4")

# Extract audio as WAV
!ffmpeg -i "{input_video}" -vn -ac 2 -ar 44100 -c:a pcm_s16le "{output_audio}" -y

# Extract video (with original audio or without depending on your need)
# Here, extracting video with original audio intact
!ffmpeg -i "{input_video}" -c copy -an "{output_video}" -y

print("Separation completed!")
print("Audio file:", output_audio)
print("Video file:", output_video)


ffmpeg version 4.4.2-0ubuntu0.22.04.1 Copyright (c) 2000-2021 the FFmpeg developers
  built with gcc 11 (Ubuntu 11.2.0-19ubuntu1)
  configuration: --prefix=/usr --extra-version=0ubuntu0.22.04.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --enable-gnutls --enable-ladspa --enable-libaom --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libdav1d --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libjack --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librabbitmq --enable-librubberband --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libsrt --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvidstab --enable-libvorbis --enable-libvpx --enab

In [3]:
import os
from pathlib import Path

# Create the output directory
output_dir = "/kaggle/working/DEMUCS"
os.makedirs(output_dir, exist_ok=True)

# Input audio file path
input_audio = "/kaggle/working/OUTPUT/output_audio.wav"  # Change this to your actual input path

# Run demucs and specify the output folder
!demucs --out={output_dir} {input_audio}

print("Demucs separation completed! Outputs are in", output_dir)


Important: the default model was recently changed to `htdemucs` the latest Hybrid Transformer Demucs model. In some cases, this model can actually perform worse than previous models. To get back the old default model use `-n mdx_extra_q`.
Downloading: "https://dl.fbaipublicfiles.com/demucs/hybrid_transformer/955717e8-8726e21a.th" to /root/.cache/torch/hub/checkpoints/955717e8-8726e21a.th
100%|███████████████████████████████████████| 80.2M/80.2M [00:00<00:00, 209MB/s]
Selected model is a bag of 1 models. You will see that many progress bars per track.
Separated tracks will be stored in /kaggle/working/DEMUCS/htdemucs
Separating track /kaggle/working/OUTPUT/output_audio.wav
100%|██████████████████████████████████████████████| 40.949999999999996/40.949999999999996 [00:06<00:00,  6.16seconds/s]
Demucs separation completed! Outputs are in /kaggle/working/DEMUCS


In [1]:
!pip install resemble-enhance --upgrade

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 18.1 MB/s eta 0:00:00a 0:00:01
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.2/61.2 kB 4.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.4/60.4 kB 5.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.6/57.6 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.1/57.1 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.5/16.5 MB 90.0 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 253.7/253.7 kB 21.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.6/11.6 MB 88.7 MB/s eta 0:00:00:00:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.2/18.2 MB 85.5 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.2/12.2 MB 103.3 MB/s eta 0:00:0000:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 76.7 MB/s eta 0:00

In [3]:
!resemble-enhance /kaggle/input/dubingdata /kaggle/working

[2025-10-14 09:34:42,399] [INFO] [real_accelerator.py:161:get_accelerator] Setting ds_accelerator to cuda (auto detect)
Processing /kaggle/working/bass (1).wav:   0%|            | 0/6 [00:00<?, ?it/s]Cloning into '/usr/local/lib/python3.11/dist-packages/resemble_enhance/model_repo'...
remote: Enumerating objects: 40, done.
remote: Counting objects: 100% (39/39), done.
remote: Compressing objects: 100% (37/37), done.
remote: Total 40 (delta 12), reused 0 (delta 0), pack-reused 1 (from 1)
Unpacking objects: 100% (40/40), 5.47 KiB | 350.00 KiB/s, done.
100%|█████████████████████████████████████████████| 2/2 [00:17<00:00,  8.94s/it]
Processing /kaggle/working/output_audio (2).wav:  50%|▌| 3/6 [01:11<01:05, 21.79
100%|█████████████████████████████████████████████| 2/2 [00:18<00:00,  9.27s/it]
Processing /kaggle/working/recombined_mix.wav:  67%|▋| 4/6 [01:29<00:41, 20.60s/
Processing /kaggle/working/vocals (3).wav: 100%|██| 6/6 [02:10<00:00, 21.68s/it]
🌟 Enhancement done! 6 files processed i

In [ ]:
# import os
# import torch
# import torchaudio

# # Load Silero VAD
# model, utils = torch.hub.load(
#     repo_or_dir='snakers4/silero-vad',
#     model='silero_vad',
#     force_reload=True,
#     trust_repo=True
# )
# (get_speech_timestamps, save_audio, read_audio, VADIterator, collect_chunks) = utils

# audio_path = "/kaggle/working/vocals.wav"

# # Create output folder
# vad_folder = "/kaggle/working/VAD"
# os.makedirs(vad_folder, exist_ok=True)

# # Load audio
# waveform, sr = torchaudio.load(audio_path)

# # Convert to mono if stereo
# if waveform.shape[0] > 1:
#     waveform = waveform.mean(dim=0, keepdim=True)

# # Resample to 16kHz
# target_sr = 16000
# if sr != target_sr:
#     resampler = torchaudio.transforms.Resample(sr, target_sr)
#     waveform = resampler(waveform)
#     sr = target_sr

# # Run VAD
# speech_timestamps = get_speech_timestamps(waveform.squeeze(), model, sampling_rate=sr)

# print("Detected speech segments:")
# for seg in speech_timestamps:
#     print(f"Start: {seg['start']/sr:.2f}s, End: {seg['end']/sr:.2f}s")

# # Save each detected speech segment into VAD folder
# for idx, seg in enumerate(speech_timestamps):
#     chunk = waveform[:, seg['start']:seg['end']]  # shape: [1, samples]

#     # Ensure it's always [channels, samples]
#     if chunk.dim() == 1:
#         chunk = chunk.unsqueeze(0)

#     out_path = os.path.join(vad_folder, f"speech_segment_{idx}.wav")
#     torchaudio.save(out_path, chunk, sample_rate=sr)
#     print(f"Saved segment {idx} -> {out_path}")


In [6]:
import os
import json
import torch
import torchaudio
from pydub import AudioSegment, silence

# --------------------------
# Function to split long segments
# --------------------------
def split_if_too_long(audio_path, start_ms, end_ms, max_len=10, min_silence_len=200, silence_thresh=-40):
    """
    Split a segment only if it's longer than max_len (in seconds).
    Otherwise, keep it as is.
    Returns list of (start_ms, end_ms).
    """
    audio = AudioSegment.from_file(audio_path)
    segment_audio = audio[start_ms:end_ms]
    duration = len(segment_audio)

    if duration <= max_len * 1000:
        return [(start_ms, end_ms)]

    # Try to find silence near midpoint
    midpoint = duration // 2
    search_window = segment_audio[max(0, midpoint - 3000):min(duration, midpoint + 3000)]

    silences = silence.detect_silence(
        search_window,
        min_silence_len=min_silence_len,
        silence_thresh=silence_thresh
    )

    if silences:
        # Pick silence closest to midpoint
        silence_mid = min(silences, key=lambda s: abs((s[0]+s[1])//2 - midpoint))
        cut_point = max(0, midpoint - 3000) + (silence_mid[0] + silence_mid[1]) // 2
    else:
        # If no silence, just hard cut at midpoint
        cut_point = midpoint

    # Return two chunks (recursively split further if needed)
    left = split_if_too_long(audio_path, start_ms, start_ms + cut_point, max_len, min_silence_len, silence_thresh)
    right = split_if_too_long(audio_path, start_ms + cut_point, end_ms, max_len, min_silence_len, silence_thresh)
    return left + right


# --------------------------
# Main VAD pipeline
# --------------------------
# Load Silero VAD
model, utils = torch.hub.load(
    repo_or_dir='snakers4/silero-vad',
    model='silero_vad',
    force_reload=True,
    trust_repo=True
)
(get_speech_timestamps, save_audio, read_audio, VADIterator, collect_chunks) = utils

audio_path = "/kaggle/input/dubingdata/vocals (3).wav"

# Create output folder
vad_folder = "/kaggle/working/VAD"
os.makedirs(vad_folder, exist_ok=True)

# Load audio
waveform, sr = torchaudio.load(audio_path)

# Convert to mono if stereo
if waveform.shape[0] > 1:
    waveform = waveform.mean(dim=0, keepdim=True)

# Resample to 16kHz
target_sr = 16000
if sr != target_sr:
    resampler = torchaudio.transforms.Resample(sr, target_sr)
    waveform = resampler(waveform)
    sr = target_sr

# Run VAD
speech_timestamps = get_speech_timestamps(waveform.squeeze(), model, sampling_rate=sr)

print("Detected speech segments (before splitting):")
for seg in speech_timestamps:
    print(f"Start: {seg['start']/sr:.2f}s, End: {seg['end']/sr:.2f}s")

# Save final segments (after splitting) + JSON
all_segments = []
segment_counter = 0
audio_full = AudioSegment.from_file(audio_path)

for seg in speech_timestamps:
    start_ms = int(seg['start'] / sr * 1000)
    end_ms = int(seg['end'] / sr * 1000)

    # Split if too long
    sub_segments = split_if_too_long(audio_path, start_ms, end_ms, max_len=10)

    for (s, e) in sub_segments:
        chunk = audio_full[s:e]
        out_path = os.path.join(vad_folder, f"speech_segment_{segment_counter}.wav")
        chunk.export(out_path, format="wav")

        all_segments.append({
            "id": segment_counter,
            "file": out_path,
            "start": s / 1000.0,
            "end": e / 1000.0
        })

        print(f"💾 Saved segment {segment_counter} -> {out_path} ({(e-s)/1000:.2f}s)")
        segment_counter += 1

# Save JSON metadata
json_path = os.path.join(vad_folder, "segments.json")
with open(json_path, "w") as f:
    json.dump(all_segments, f, indent=4)

print(f"\n📑 JSON saved -> {json_path}")


Downloading: "https://github.com/snakers4/silero-vad/zipball/master" to /root/.cache/torch/hub/master.zip


Detected speech segments (before splitting):
Start: 0.51s, End: 3.45s
Start: 4.10s, End: 5.76s
Start: 6.05s, End: 12.13s
Start: 12.58s, End: 13.50s
Start: 13.83s, End: 15.13s
Start: 15.39s, End: 21.18s
Start: 21.60s, End: 24.29s
Start: 24.80s, End: 27.29s
Start: 27.55s, End: 31.42s
Start: 31.52s, End: 34.91s
Start: 35.27s, End: 37.44s
Start: 37.83s, End: 38.72s
Start: 38.88s, End: 39.69s
💾 Saved segment 0 -> /kaggle/working/VAD/speech_segment_0.wav (2.94s)
💾 Saved segment 1 -> /kaggle/working/VAD/speech_segment_1.wav (1.66s)
💾 Saved segment 2 -> /kaggle/working/VAD/speech_segment_2.wav (6.08s)
💾 Saved segment 3 -> /kaggle/working/VAD/speech_segment_3.wav (0.92s)
💾 Saved segment 4 -> /kaggle/working/VAD/speech_segment_4.wav (1.31s)
💾 Saved segment 5 -> /kaggle/working/VAD/speech_segment_5.wav (5.79s)
💾 Saved segment 6 -> /kaggle/working/VAD/speech_segment_6.wav (2.68s)
💾 Saved segment 7 -> /kaggle/working/VAD/speech_segment_7.wav (2.49s)
💾 Saved segment 8 -> /kaggle/working/VAD/speech_s

In [2]:
import os

# Input and output directories
vad_dir = "/kaggle/working/VAD"
output_dir = "/kaggle/working/translated"
os.makedirs(output_dir, exist_ok=True)

# Target language
tgt_lang = "deu"

# Loop through all VAD segments
for file in sorted(os.listdir(vad_dir)):
    if file.endswith(".wav"):
        in_file = os.path.join(vad_dir, file)
        out_file = os.path.join(output_dir, file)  # same name as input

        print(f"🔹 Translating {file} -> {out_file}")

        # Run Seamless Expressive translation
        !expressivity_predict {in_file} --tgt_lang {tgt_lang} \
            --model_name seamless_expressivity --vocoder_name vocoder_pretssel \
            --gated-model-dir SeamlessExpressive --output_path {out_file}

print("\n✅ All translated files saved in:", output_dir)


🔹 Translating speech_segment_0.wav -> /kaggle/working/translated/speech_segment_0.wav
Using the cached tokenizer of seamless_expressivity. Set `force` to `True` to download again.
Using the cached tokenizer of seamless_expressivity. Set `force` to `True` to download again.
Using the cached tokenizer of seamless_expressivity. Set `force` to `True` to download again.
/usr/local/lib/python3.11/dist-packages/torch/nn/utils/weight_norm.py:28: UserWarning: torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.
  warnings.warn("torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.")
🔹 Translating speech_segment_1.wav -> /kaggle/working/translated/speech_segment_1.wav
Using the cached tokenizer of seamless_expressivity. Set `force` to `True` to download again.
Using the cached tokenizer of seamless_expressivity. Set `force` to `True` to download again.
Using the cached tokenizer of seamless_expressivity. Set

In [3]:
import json
import os
from pydub import AudioSegment

# ========= CONFIG =========
vad_json = "/kaggle/working/VAD/segments.json"   # JSON with VAD timings
vad_folder = "/kaggle/working/VAD"                   # Original VAD segments folder
translated_folder = "/kaggle/working/translated"     # Translated audio segments folder
output_path = "/kaggle/working/final_aligned.wav"    # Final output

# ========= LOAD VAD TIMESTAMPS =========
with open(vad_json, "r") as f:
    vad_segments = json.load(f)

final_audio = AudioSegment.silent(duration=0)

for seg in vad_segments:
    idx = seg["id"]  # assuming JSON has {"id": 0, "start": .., "end": ..}
    start_ms = int(seg["start"] * 1000)
    end_ms = int(seg["end"] * 1000)
    duration_ms = end_ms - start_ms

    translated_path = os.path.join(translated_folder, f"speech_segment_{idx}.wav")
    if not os.path.exists(translated_path):
        print(f"⚠️ Missing translated file for seg {idx}, skipping...")
        continue

    seg_audio = AudioSegment.from_file(translated_path).set_channels(1).set_frame_rate(16000)

    # ===== Match duration =====
    if len(seg_audio) > duration_ms:
        seg_audio = seg_audio[:duration_ms]  # trim
        print(f"✂️ Trimmed seg {idx} to {duration_ms/1000:.2f}s")
    elif len(seg_audio) < duration_ms:
        silence_padding = AudioSegment.silent(duration=duration_ms - len(seg_audio))
        seg_audio = seg_audio + silence_padding
        print(f"➕ Padded seg {idx} to {duration_ms/1000:.2f}s")

    # ===== Insert gap if needed =====
    current_length = len(final_audio)
    if current_length < start_ms:
        gap = AudioSegment.silent(duration=start_ms - current_length)
        final_audio += gap
        print(f"🕐 Inserted silence before seg {idx}: {(start_ms - current_length)/1000:.2f}s")

    # ===== Append aligned segment =====
    final_audio += seg_audio
    print(f"✅ Added seg {idx} ({duration_ms/1000:.2f}s)")

# ========= EXPORT =========
final_audio.export(output_path, format="wav")
print(f"\n🎉 Final aligned audio saved at: {output_path}")


✂️ Trimmed seg 0 to 2.94s
🕐 Inserted silence before seg 0: 0.51s
✅ Added seg 0 (2.94s)
🕐 Inserted silence before seg 1: 0.64s
✅ Added seg 1 (1.66s)
✂️ Trimmed seg 2 to 6.08s
🕐 Inserted silence before seg 2: 0.29s
✅ Added seg 2 (6.08s)
✂️ Trimmed seg 3 to 0.92s
🕐 Inserted silence before seg 3: 0.45s
✅ Added seg 3 (0.92s)
➕ Padded seg 4 to 1.31s
🕐 Inserted silence before seg 4: 0.32s
✅ Added seg 4 (1.31s)
➕ Padded seg 5 to 5.79s
🕐 Inserted silence before seg 5: 0.26s
✅ Added seg 5 (5.79s)
✂️ Trimmed seg 6 to 2.68s
🕐 Inserted silence before seg 6: 0.42s
✅ Added seg 6 (2.68s)
✂️ Trimmed seg 7 to 2.49s
🕐 Inserted silence before seg 7: 0.52s
✅ Added seg 7 (2.49s)
✂️ Trimmed seg 8 to 3.87s
🕐 Inserted silence before seg 8: 0.26s
✅ Added seg 8 (3.87s)
✂️ Trimmed seg 9 to 3.39s
🕐 Inserted silence before seg 9: 0.10s
✅ Added seg 9 (3.39s)
✂️ Trimmed seg 10 to 2.17s
🕐 Inserted silence before seg 10: 0.36s
✅ Added seg 10 (2.17s)
➕ Padded seg 11 to 0.89s
🕐 Inserted silence before seg 11: 0.39s
✅ Add

In [7]:
!pip install openvoice-cli

INFO: pip is looking at multiple versions of dtw-python to determine which version is compatible with other requirements. This could take a while.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.1/48.1 kB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.8/44.8 kB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 213.1/213.1 kB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 27.6 MB/s eta 0:00:0000:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.9/53.9 kB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.2/40.2 MB 43.4 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.6/38.6 MB 45.5 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.4/17.4 MB 87.3 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 801.7/801.7 kB 31.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━

In [5]:
from pydub import AudioSegment

# Load separated audio files
vocals = AudioSegment.from_file("/kaggle/working/cloned_output.wav")
bass = AudioSegment.from_file("/kaggle/working/DEMUCS/htdemucs/output_audio/bass.wav")
drums = AudioSegment.from_file("/kaggle/working/DEMUCS/htdemucs/output_audio/drums.wav")
other = AudioSegment.from_file("/kaggle/working/DEMUCS/htdemucs/output_audio/other.wav")

# Ensure all have same length
min_len = min(len(vocals), len(bass), len(drums), len(other))
vocals = vocals[:min_len]
bass = bass[:min_len]
drums = drums[:min_len]
other = other[:min_len]

# Combine (mix) all stems together
final_mix = vocals.overlay(bass).overlay(drums).overlay(other)

# Export final mix
final_mix.export("/kaggle/working/recombined_mix.wav", format="wav")
print("✅ Recombined audio saved as 'recombined_mix.wav'")


✅ Recombined audio saved as 'recombined_mix.wav'


In [4]:
!python -m openvoice_cli single \
  -i /kaggle/working/final_aligned.wav \
  -r /kaggle/working/OUTPUT/output_audio.wav \
  -o /kaggle/working/cloned_output.wav \
  -d cuda

/usr/local/lib/python3.11/dist-packages/torch/nn/utils/weight_norm.py:28: UserWarning: torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.
  warnings.warn("torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.")
Loaded checkpoint '/usr/local/lib/python3.11/dist-packages/openvoice_cli/checkpoints/converter/checkpoint.pth'
missing/unexpected keys: [] []
OpenVoice version: v2
[(0.046, 20.594), (21.166, 39.6903125)]
after vad: dur = 39.072
/usr/local/lib/python3.11/dist-packages/torch/functional.py:660: UserWarning: stft with return_complex=False is deprecated. In a future pytorch release, stft will return complex tensors for all inputs, and return_complex=False will raise an error.
Note: you can still call torch.view_as_real on the complex output to recover the old return format. (Triggered internally at ../aten/src/ATen/native/SpectralOps.cpp:874.)
  return _VF.stft(input, n_fft, hop_length, win_len

In [9]:
!git clone https://github.com/justinjohn0306/Wav2Lip.git


Cloning into 'Wav2Lip'...
remote: Enumerating objects: 534, done.
remote: Total 534 (delta 0), reused 0 (delta 0), pack-reused 534 (from 1)
Receiving objects: 100% (534/534), 29.78 MiB | 36.30 MiB/s, done.
Resolving deltas: 100% (272/272), done.


In [6]:
%cd Wav2Lip

/kaggle/working/Wav2Lip


In [11]:
!pip install numpy>=1.25 scipy>=1.12 torch>=2.0 torchvision>=0.15 opencv-python>=4.10 tqdm numba ffmpeg-python


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
fairseq2n 0.2.1 requires torch==2.2.2, but you have torch 2.1.1 which is incompatible.
resemble-enhance 0.0.1 requires librosa==0.10.1, but you have librosa 0.9.1 which is incompatible.
resemble-enhance 0.0.1 requires torchaudio==2.1.1, but you have torchaudio 2.2.2 which is incompatible.
resemble-enhance 0.0.1 requires tqdm==4.66.1, but you have tqdm 4.64.1 which is incompatible.
torchaudio 2.2.2 requires torch==2.2.2, but you have torch 2.1.1 which is incompatible.


In [12]:
# Step 2: Download pretrained model (standard Wav2Lip)
!mkdir -p checkpoints
!wget 'https://github.com/justinjohn0306/Wav2Lip/releases/download/models/wav2lip_gan.pth' -O 'checkpoints/wav2lip_gan.pth'


--2025-10-13 11:49:57--  https://github.com/justinjohn0306/Wav2Lip/releases/download/models/wav2lip_gan.pth
Resolving github.com (github.com)... 140.82.112.4
Connecting to github.com (github.com)|140.82.112.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://release-assets.githubusercontent.com/github-production-release-asset/615543729/76281b9f-48b8-4cbf-9a05-edf61d847109?sp=r&sv=2018-11-09&sr=b&spr=https&se=2025-10-13T12%3A50%3A03Z&rscd=attachment%3B+filename%3Dwav2lip_gan.pth&rsct=application%2Foctet-stream&skoid=96c2d410-5711-43a1-aedd-ab1947aa7ab0&sktid=398a6654-997b-47e9-b12b-9515b896b4de&skt=2025-10-13T11%3A49%3A10Z&ske=2025-10-13T12%3A50%3A03Z&sks=b&skv=2018-11-09&sig=oF5aZh1YQN0LAQ%2BSNeb%2FlIGt1MY%2BA1pfv5EBVEeS8N0%3D&jwt=eyJ0eXAiOiJKV1QiLCJhbGciOiJIUzI1NiJ9.eyJpc3MiOiJnaXRodWIuY29tIiwiYXVkIjoicmVsZWFzZS1hc3NldHMuZ2l0aHVidXNlcmNvbnRlbnQuY29tIiwia2V5Ijoia2V5MSIsImV4cCI6MTc2MDM1OTc5NywibmJmIjoxNzYwMzU2MTk3LCJwYXRoIjoicmVsZWFzZWFzc2V0cHJvZHVjd

In [7]:
# Step 3: Define your paths (your project files)
VIDEO_PATH = "/kaggle/working/OUTPUT/output_video.mp4"
TRANSLATED_AUDIO_PATH = "/kaggle/working/recombined_mix.wav"
OUTPUT_VIDEO_PATH = "/kaggle/working/lip_synced_video_complete_audio.mp4"

In [14]:
!pip install batch-face==1.5.3


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.6/30.6 MB 50.7 MB/s eta 0:00:00:00:010:01m


In [15]:
!wget https://github.com/justinjohn0306/Wav2Lip/releases/download/models/mobilenet.pth -O checkpoints/mobilenet.pth


--2025-10-13 11:50:08--  https://github.com/justinjohn0306/Wav2Lip/releases/download/models/mobilenet.pth
Resolving github.com (github.com)... 140.82.114.4
Connecting to github.com (github.com)|140.82.114.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://release-assets.githubusercontent.com/github-production-release-asset/615543729/0f1702ef-4998-4acd-abc8-b80c52e838b9?sp=r&sv=2018-11-09&sr=b&spr=https&se=2025-10-13T12%3A37%3A53Z&rscd=attachment%3B+filename%3Dmobilenet.pth&rsct=application%2Foctet-stream&skoid=96c2d410-5711-43a1-aedd-ab1947aa7ab0&sktid=398a6654-997b-47e9-b12b-9515b896b4de&skt=2025-10-13T11%3A37%3A46Z&ske=2025-10-13T12%3A37%3A53Z&sks=b&skv=2018-11-09&sig=L%2BXxUVbCtCw3Vg13L%2BS8JKtwJyEl3ezbPCcOr0oVrHo%3D&jwt=eyJ0eXAiOiJKV1QiLCJhbGciOiJIUzI1NiJ9.eyJpc3MiOiJnaXRodWIuY29tIiwiYXVkIjoicmVsZWFzZS1hc3NldHMuZ2l0aHVidXNlcmNvbnRlbnQuY29tIiwia2V5Ijoia2V5MSIsImV4cCI6MTc2MDM1NjUwOCwibmJmIjoxNzYwMzU2MjA4LCJwYXRoIjoicmVsZWFzZWFzc2V0cHJvZHVjdGlvbi5

In [8]:
!python inference.py \
    --checkpoint_path checkpoints/wav2lip_gan.pth \
    --face $VIDEO_PATH \
    --audio $TRANSLATED_AUDIO_PATH \
    --outfile $OUTPUT_VIDEO_PATH \
    --nosmooth


/usr/local/lib/python3.11/dist-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: '/usr/local/lib/python3.11/dist-packages/torchvision/image.so: undefined symbol: _ZN3c1017RegisterOperatorsD1Ev'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `libpng` installed before building `torchvision` from source?
  warn(
Using cuda for inference.
Load checkpoint from: checkpoints/wav2lip_gan.pth
Models loaded
Reading video frames...
Number of frames available for inference: 952
(80, 3175)
Length of mel chunks: 949
100%|█████████████████████████████████████████████| 8/8 [00:12<00:00,  1.57s/it]
wav2lip prediction time: 12.59143877029419
ffmpeg version 4.4.2-0ubuntu0.22.04.1 Copyright (c) 2000-2021 the FFmpeg developers
  built with gcc 11 (Ubuntu 11.2.0-19ubuntu1)
  configuration: --prefix=/usr --extra-version=0ubuntu0.2

In [14]:
import os
import zipfile

# Path to your Kaggle output directory
output_dir = "/kaggle/working"  # change if needed

# List of folders you want to include
folders_to_zip = ["DEMUX", "OUTPUT", "VAD", "translated"]

# List of individual files you want to include
files_to_zip = ["recombined_mix.wav", "bass.wav", "other.wav", "vocals.wav", "drums.wav",
                "cloned_output.wav", "final_aligned.wav", "lip_synced_video_complete_audio.mp4"]

# Name of the zip file to create (explicitly in /kaggle/working)
zip_filename = os.path.join(output_dir, "deu.zip")

with zipfile.ZipFile(zip_filename, 'w', zipfile.ZIP_DEFLATED) as zipf:
    # Add folders
    for folder_name in folders_to_zip:
        folder_path = os.path.join(output_dir, folder_name)
        for root, dirs, files in os.walk(folder_path):
            for file in files:
                file_path = os.path.join(root, file)
                zipf.write(file_path, os.path.relpath(file_path, output_dir))
    
    # Add individual files
    for file_name in files_to_zip:
        file_path = os.path.join(output_dir, file_name)
        if os.path.exists(file_path):
            zipf.write(file_path, os.path.relpath(file_path, output_dir))
        else:
            print(f"Warning: {file_name} does not exist in {output_dir}")

print(f"Zip file created at: {zip_filename}")


Zip file created at: /kaggle/working/deu.zip


In [28]:
!pip install realesrgan
!pip install facexlib  # optional, for face enhancement


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 172.5/172.5 kB 3.9 MB/s eta 0:00:00a 0:00:01
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.8/46.8 kB 2.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 178.0/178.0 kB 10.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.6/59.6 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.2/52.2 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 299.6/299.6 kB 17.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 74.1 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 256.2/256.2 kB 11.5 MB/s eta 0:00:00
  Created wheel for basicsr: filename=basicsr-1.4.2-py3-none-any.whl size=214817 sha256=25c3ce64e7ec96a1e5df500379fc5a7742e9a2035b8972b0d169a7faf038ad07
  Stored in directory: /root/.cache/pip/wheels/6d/a4/b3/9f888ba88efcae6dd4bbce69832363de9c4051142674f779fa
 

In [9]:
!cd ..

In [10]:
!cd /kaggle/working/

In [32]:
!git clone https://github.com/xinntao/Real-ESRGAN.git
!pip install -r requirements.txt


Cloning into 'Real-ESRGAN'...
remote: Enumerating objects: 759, done.
remote: Counting objects: 100% (121/121), done.
remote: Compressing objects: 100% (23/23), done.
remote: Total 759 (delta 106), reused 98 (delta 98), pack-reused 638 (from 1)
Receiving objects: 100% (759/759), 5.38 MiB | 24.81 MiB/s, done.
Resolving deltas: 100% (415/415), done.
/kaggle/working/Wav2Lip/Real-ESRGAN


In [11]:
%cd Real-ESRGAN


/kaggle/working/Wav2Lip/Real-ESRGAN


In [34]:
%cd /kaggle/working/Wav2Lip/Real-ESRGAN
!pip install -e .

/kaggle/working/Wav2Lip/Real-ESRGAN
Obtaining file:///kaggle/working/Wav2Lip/Real-ESRGAN
  Preparing metadata (setup.py) ... done
  Attempting uninstall: realesrgan
    Found existing installation: realesrgan 0.3.0
    Uninstalling realesrgan-0.3.0:
      Successfully uninstalled realesrgan-0.3.0
  Running setup.py develop for realesrgan


In [12]:
!ffmpeg -i /kaggle/working/lip_synced_video_complete_audio.mp4 \
-c:v libx264 -preset slow -crf 18 -c:a copy \
/kaggle/working/lip_synced_video4_h264.mp4


ffmpeg version 4.4.2-0ubuntu0.22.04.1 Copyright (c) 2000-2021 the FFmpeg developers
  built with gcc 11 (Ubuntu 11.2.0-19ubuntu1)
  configuration: --prefix=/usr --extra-version=0ubuntu0.22.04.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --enable-gnutls --enable-ladspa --enable-libaom --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libdav1d --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libjack --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librabbitmq --enable-librubberband --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libsrt --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvidstab --enable-libvorbis --enable-libvpx --enab

In [13]:
!python /kaggle/working/Wav2Lip/Real-ESRGAN/inference_realesrgan_video.py \
    -i /kaggle/working/lip_synced_video4_h264.mp4 \
    -n realesr-animevideov3 \
    -s 2 \
    --face_enhance \
    -o /kaggle/working/Wav2Lip/Real-ESRGAN/output_upscaled


/usr/local/lib/python3.11/dist-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: '/usr/local/lib/python3.11/dist-packages/torchvision/image.so: undefined symbol: _ZN3c1017RegisterOperatorsD1Ev'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `libpng` installed before building `torchvision` from source?
  warn(
/usr/local/lib/python3.11/dist-packages/torchvision/transforms/functional_tensor.py:5: UserWarning: The torchvision.transforms.functional_tensor module is deprecated in 0.15 and will be **removed in 0.17**. Please don't rely on it. You probably just need to use APIs in torchvision.transforms.functional or in torchvision.transforms.v2.functional.
  warnings.warn(
inference:   0%|                                   | 0/2 [00:00<?, ?sub_video/s]/usr/local/lib/python3.11/dist-packages/torchvision/io/image.py